In [1]:
# dont forget to accept license terms at https://huggingface.co/pyannote/speaker-diarization
# and at https://huggingface.co/pyannote/segmentation
# and at https://huggingface.co/pyannote/speaker-diarization-3.1
# and at https://huggingface.co/pyannote/segmentation-3.0
import os

hugging_face_token = str(os.getenv('HF_TOKEN'))
#print(hugging_face_token)

In [ ]:
# to install pyannote
!pip install pyannote.audio

In [10]:
# to install speechbrain
!pip install speechbrain

In [5]:
# diarization
# reference (with results on AMI) https://github.com/pyannote/pyannote-audio?tab=readme-ov-file

from pyannote.audio import Pipeline
import speechbrain

# "pyannote/speaker-diarization@2.1"
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1",
                                    use_auth_token=hugging_face_token)
print(1)
audio_input = '/home/kozi/Documents/_onlab_git/ami/ami_dataset/test/audio/Array1-01/EN2002a.Array1-01.wav'
# apply the pipeline to an audio file
diarization = pipeline(audio_input, num_speakers = 4)
print(2)

# dump the diarization output to disk using RTTM format
with open("/home/kozi/Documents/_onlab_git/ami/audio-test-pyannote.rttm", "w") as rttm:
    diarization.write_rttm(rttm)

1
2


In [21]:
from typing import TextIO, Optional, Iterator, Text

class Annotation(Annotation):
    @classmethod
    def from_rttm(
        cls, rttm_file: TextIO, uri: Optional[str] = None, modality: Optional[str] = None,
    ) -> "Annotation":
        """Create annotation from rttmParameters
        ----------
        rttm_file : string,
            path to the rttm file
        uri : string, optional
            name of annotated resource (e.g. audio or video file)
        modality : string, optional
            name of annotated modality
        Returns
        -------
        annotation : Annotation
            New annotation
        """
        segment_list = []
        for line in rttm_file:
            line = line.rstrip().split(" ")
            segment_list.append(
                (
                    Segment(start=float(line[3]), end=float(line[3]) + float(line[4])),
                    int(line[2]),
                    str(line[7]),
                )
            )
        return Annotation.from_records(segment_list, uri, modality)
    def _iter_audacity(self) -> Iterator[Text]:
        """Generate lines for a audacity marker file for this annotation
        Returns
        -------
        iterator: Iterator[str]
            An iterator over audacity text lines
        """
        for segment, _, label in self.itertracks(yield_label=True):
            yield f"{segment.start:.3f}\t{segment.start + segment.duration:.3f}\t{label}\n"
    def to_audacity(self) -> Text:
        """Serialize annotation as a string using Audacity format
        Returns
        -------
        serialized: str
            audacity marker string
        """
        return "".join([line for line in self._iter_audacity()])
    def write_audacity(self, file: TextIO):
        """Dump annotation to file using Audacity format
        Parameters
        ----------
        file : file object
        Usage
        -----
        >>> with open('file.txt', 'w') as file:
        ...     annotation.write_audacity(file)
        """
        for line in self._iter_audacity():
            file.write(line)
    @classmethod
    def from_audacity(
        cls, audacity_file: str, uri: Optional[str] = None, modality: Optional[str] = None,
    ) -> "Annotation":
        """Create annotation from rttm file
        Parameters
        ----------
        audacity_txt_file : string,
            path to the rttm file
        uri : string, optional
            name of annotated resource (e.g. audio or video file)
        modality : string, optional
            name of annotated modality
        Returns
        -------
        annotation : Annotation
            New annotation
        """
        segment_list = []
        for line in audacity_file:
            start, end, label = line.rstrip().split("\t")
            segment_list.append(
                (Segment(start=float(start), end=float(end)), 1, str(label))
            )
        return Annotation.from_records(segment_list, uri, modality)

In [22]:
# okay this tutorial and file is unusable
# calculate DER metrics
from pyannote.metrics.diarization import DiarizationErrorRate

with open('/home/kozi/Documents/_onlab_git/ami/ami_dataset/test/rttm/EN2002a.rttm') as f :
    ref = Annotation.from_audacity(f)
with open('/home/kozi/Documents/_onlab_git/ami/audio-test-pyannote.rttm') as f :
    hyp = Annotation.from_rttm(f)

#ref = '/home/kozi/Documents/_onlab_git/ami/ami_dataset/test/rttm/EN2002a.rttm'
#hyp = '/home/kozi/Documents/_onlab_git/ami/audio-test-pyannote.rttm'
der = DiarizationErrorRate()                                                                                          
der_result = der(ref, hyp)                                                                                                                    
print(f'DER: {der_result:.2f}')         

der = DiarizationErrorRate()

der_result = der(ref,hyp)

ValueError: not enough values to unpack (expected 3, got 1)

In [23]:
!python3 /home/kozi/Documents/_onlab_git/dscore/score.py -s '/home/kozi/Documents/_onlab_git/ami/audio-test-pyannote.rttm' -r "/home/kozi/Documents/_onlab_git/ami/ami_dataset/test/rttm/EN2002a.rttm" # system outputs and reference

Loading speaker turns from reference RTTMs...
Loading speaker turns from system RTTMs...
Trimming reference speaker turns to UEM scoring regions...
Trimming system speaker turns to UEM scoring regions...
Checking for overlapping reference speaker turns...
Checking for overlapping system speaker turns...
Scoring...
File                  DER     JER    B3-Precision    B3-Recall    B3-F1    GKT(ref, sys)    GKT(sys, ref)    H(ref|sys)    H(sys|ref)    MI    NMI
-----------------  ------  ------  --------------  -----------  -------  ---------------  ---------------  ------------  ------------  ----  -----
EN2002a              0.00  100.00            0.14         1.00     0.24             1.00             0.00          3.26          0.00  0.00   0.00
EN2002a.Array1-01  100.00  100.00            1.00         0.24     0.38             0.00             1.00          0.00          2.60  0.00   0.00
*** OVERALL ***      0.00  100.00            0.57         0.62     0.59             0.45        

Well... 100% is unlikely (NeMo is supposed to overperform pyannote).